In [ ]:
import pygame
import sys
from utils import set_up_clients, create_message, generate_image, generate_and_play_audio
from models import User, NPC, Location
import threading

import requests
import io
import logging

logging.basicConfig(level=logging.INFO)

# Initialize clients
anthropic_client, openai_client, elevenlabs_client = set_up_clients()

# Game constants
WIDTH, HEIGHT = 1280, 720
IMAGE_WIDTH = 720
CHAT_WIDTH = WIDTH - IMAGE_WIDTH
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

In [ ]:
# Game state
game_state = {
    "player": None,
    "current_location": "",
    "current_thing_to_show": None,
    "npcs": [],
    "locations": [],
    "historical_events": "",
    "game_progression": "Game started",
}
game_intro = ""


In [ ]:
background_story = "Ancient Rome, just before the murder of Julius Caesar. The main character is Brutus and he is taking part in the intrigue with other senators. The game is starting with the conversation of Julius Caesar and Brutus. Julius Caesar asks Brutus, 'How often do you think about the Roman Empire?' and Brutus has to answer."

tool_begin_game = [
    {
        "name": "start_game",
        "description": "Your job is to create staring setup of the game",
        "input_schema": {
            "type": "object",
            "properties": {
                "game_into_narration": {
                    "type" : "string",
                    "description": "The intro narration of the game"
                },
                "starting_location" : {
                    "type": "object",
                    "description": "The first location where the game starts in",
                    "properties":
                        {
                            "location_name" : {
                                "type" : "string"
                            },
                            "location_description" : {
                                "type" : "string"
                            }
                        }
                },
                "main_character" : {
                    "type": "object",
                    "description": "The playable character of the game",
                    "properties":
                        {
                            "character_name" : {
                                "type" : "string"
                            },
                            "character_description" : {
                                "type" : "string"
                            }
                        }
                },
                "list_of_npcs" : {
                    "type": "array",
                    "description": "All important NPCs in the game except for main character",

                    "items": {
                        "type": "object",
                        "properties":
                            {
                                "npc_name" : {
                                    "type" : "string"
                                },
                                "npc_description" : {
                                    "type" : "string"
                                }
                            }
                    }
                },
                "list_of_locations": {
                    "type": "array",
                    "description": "All important locations in the game other than starting lovcation",
                    "items": {
                        "type": "object",
                        "properties":
                            {
                                "location_name" : {
                                    "type" : "string"
                                },
                                "location_description" : {
                                    "type" : "string"
                                }
                            }
                    }
                },
                "real_history": {
                    "type": "string",
                    "description": "The real history of the events that should be recreated in the game"

                },
            },
        },
    }
]

In [ ]:
def initialize_game():
    global game_state, game_intro
    setup_result = create_message(
        anthropic_client, 
        "start_game", 
        [{"role": "user", "content": [{"type": "text", "text": background_story}]}],
        tool_begin_game
    )
    # game intro
    game_intro = setup_result['game_into_narration']
    logging.info('Game intro set')

    # game story
    game_state["historical_events"] = setup_result['real_history']
    logging.info('Game story set')

    # Create player
    try:
        player_image = generate_image(
            openai_client, 
            f"{setup_result['main_character']['character_name']} {setup_result['main_character']['character_description']} in Ancient Rome"
        )
        game_state["player"] = User(
            name=setup_result['main_character']['character_name'],
            backstory=setup_result['main_character']['character_description'], 
            image_link=player_image
        )
        logging.info(f"Player created: {setup_result['main_character']['character_name']}")
    except Exception as e:
        logging.error(f"Error creating player: {e}")

    # Create locations
    for loc in setup_result['list_of_locations']:
        try:
            loc_image = generate_image(
                openai_client, 
                f"{loc['location_name']} {loc['location_description']} in Ancient Rome"
            )
            game_state["locations"].append(Location(name=loc['location_name'], description=loc['location_description'], image_link=loc_image))
            logging.info(f"Location created: {loc['location_name']}")
        except Exception as e:
            logging.error(f"Error creating location {loc['location_name']}: {e}")

    # Create NPCs
    for npc in setup_result['list_of_npcs']:
        try:
            npc_image = generate_image(
                openai_client, 
                f"{npc['npc_name']} {npc['npc_description']}  in Ancient Rome")
            game_state["npcs"].append(
                NPC(name=npc['npc_name'], backstory=npc['npc_description'], known_npcs="", last_location="", image_link=npc_image)
            )
            logging.info(f"NPC created: {npc['npc_name']}")
        except Exception as e:
            logging.error(f"Error creating NPC {npc['npc_name']}: {e}")

    # Set initial location
    try:
        loc_image = generate_image(
            openai_client,
            f"{setup_result['starting_location']['location_name']} {setup_result['starting_location']['location_description']} in Ancient Rome"
        )
        game_state["locations"].append(Location(name=setup_result['starting_location']['location_name'], description=setup_result['starting_location']['location_description'], image_link=loc_image))
        game_state["current_location"] = game_state["locations"][-1].name
        logging.info(f"Initial location set: {setup_result['starting_location']['location_name']}")
    except Exception as e:
        logging.error(f"Error setting initial location: {e}")

In [ ]:
# Initialize game
initialize_game()

In [ ]:
# print all location names and images
for loc in game_state["locations"]:
    print(loc.name)
    print(loc.image_link)

In [ ]:
def create_game_progression_tool(game_state, game_intro):
    tool_begin_game = [
        {
            "name": "game_progression",
            "description": f"Your job is to create the progression of the game. Story: {game_intro}, Real histical events to follow: {game_state['historical_events']}. Current game progression: {game_state['game_progression']}. Main player: {game_state['player'].name} Current location: {game_state['current_location']}, all locations: {[loc.name for loc in game_state['locations']]}, all NPCs: {[npc.name for npc in game_state['npcs']]}",
            "input_schema": {
                "type": "object",
                "properties": {
                    "current_location": {
                        "type" : "string",
                        "enum": [loc.name for loc in game_state['locations']],
                        "description": "The current location history takes place in"
                    },
                    "current_thing_to_show" : {
                        "type": "string",
                        "enum": [npc.name for npc in game_state['npcs']] + [loc.name for loc in game_state['locations']],
                        "description": "The current thing to show to the player, either NPC or location"
                    },
                    "game_progression": {
                        "type": "string",
                        "description": "The current game progression that will be added to the story"
                    },
                    "current_narrator_dialog": {
                        "type": "string",
                        "description": "The dialog of the narrator to be shown to the player describing what is happening in the game and all the dialogs from npcs. Always return narration"
                    },
                },
            }
        }
    ]
    return tool_begin_game

In [ ]:
def update_game(user_input_text):
    global game_state
    system_prompt = "game_progression"
    user_input = create_message(
        anthropic_client, 
        system_prompt, 
        [{"role": "user", "content": [{"type": "text", "text": f"{game_state['game_progression']} User input: {user_input_text}"}]}],
        create_game_progression_tool(game_state, game_intro)
    )
    
    # update current location
    if user_input["current_location"]:
        game_state["current_location"] = user_input["current_location"]
        logging.info(f"Current location: {game_state['current_location']}")
    
    # update current thing to show
    if user_input["current_thing_to_show"]:
        game_state["current_thing_to_show"] = user_input["current_thing_to_show"]
        logging.info(f"Current thing to show: {game_state['current_thing_to_show']}")
    
    # update game progression
    if user_input["game_progression"]:
        game_state["game_progression"] += f" {user_input['game_progression']}"
        logging.info(f"Game progression: {game_state['game_progression']}")
    
    # return narration
    return user_input["current_narrator_dialog"]

In [ ]:
# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Time Traveler's Quest")

# Font setup
title_font = pygame.font.Font(None, 36)
chat_font = pygame.font.Font(None, 24)

def render_text(surface, text, font, color, rect, aa=False, bkg=None):
    rect = pygame.Rect(rect)
    y = rect.top
    line_spacing = -2
    font_height = font.size("Tg")[1]

    for line in text.split('\n'):
        while line:
            i = 1
            if y + font_height > rect.bottom:
                break
            while font.size(line[:i])[0] < rect.width and i < len(line):
                i += 1
            if i < len(line):
                i = line.rfind(" ", 0, i) + 1
            if bkg:
                image = font.render(line[:i], 1, color, bkg)
                image.set_colorkey(bkg)
            else:
                image = font.render(line[:i], aa, color)
            surface.blit(image, (rect.left, y))
            y += font_height + line_spacing
            line = line[i:]
        y += font_height

def play_narrator_audio(client, text):
    generate_and_play_audio(client, text)

def main_game_loop():
    global game_state
    input_active = True
    input_text = ""
    clock = pygame.time.Clock()
    last_user_input = ""
    last_narration = ""

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if input_active:
                    if event.key == pygame.K_RETURN:
                        # Update the game based on the user's input
                        last_user_input = input_text
                        last_narration = update_game(input_text)

                        # Start the audio thread for the narrator's text
                        audio_thread = threading.Thread(target=play_narrator_audio, args=(elevenlabs_client, last_narration))
                        audio_thread.start()

                        input_text = ""
                    elif event.key == pygame.K_BACKSPACE:
                        input_text = input_text[:-1]
                    else:
                        input_text += event.unicode

        # Clear the screen
        screen.fill(WHITE)

        # Display the current thing (NPC or location)
        if game_state["current_thing_to_show"]:
            thing_to_show = next((npc for npc in game_state["npcs"] if npc.name == game_state["current_thing_to_show"]), None)
            if not thing_to_show:
                thing_to_show = next((loc for loc in game_state["locations"] if loc.name == game_state["current_thing_to_show"]), None)

            if thing_to_show:
                image_response = requests.get(thing_to_show.image_link)
                image_bytes = io.BytesIO(image_response.content)
                thing_image = pygame.image.load(image_bytes)
                screen.blit(pygame.transform.scale(thing_image, (IMAGE_WIDTH, HEIGHT)), (0, 0))

        # Display the last user input and the new narration
        chat_rect = pygame.Rect(IMAGE_WIDTH, 0, CHAT_WIDTH, HEIGHT - 40)
        chat_text = f"{game_state['player'].name}: {last_user_input}\n\nNarrator: {last_narration}"
        render_text(screen, chat_text, chat_font, BLACK, chat_rect)

        # Display the input text box
        input_rect = pygame.Rect(IMAGE_WIDTH, HEIGHT - 40, CHAT_WIDTH, 40)
        pygame.draw.rect(screen, GRAY, input_rect)
        pygame.draw.rect(screen, BLACK, input_rect, 2)

        # Display the input text with a cursor
        cursor = "_" if pygame.time.get_ticks() % 1000 < 500 else " "
        render_text(screen, input_text + cursor, chat_font, BLACK, input_rect)

        # Update the display
        pygame.display.flip()
        clock.tick(30)

# Start the main game loop
main_game_loop()
